In [1]:
#import modules and set up environment 
import os
import sys
path = "../../src/"

sys.path.append(path)

sys.path.append(path)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import jax.numpy as jnp
from jax import jit, grad, vmap, hessian, scipy, random

#import sgmcmc code 
import models.logistic_regression.logistic_regression as lr
import samplers.sgld as sgld
import samplers.sgldps as sgldps
import samplers.sgd as sgd

key = random.PRNGKey(0)

In [2]:
#set up model-specific gradient functions
gradf_0 = lr.gradf_0
gradf_i_batch = lr.gradf_i_batch
post_var = lr.post_var

### Set up model and data

In [3]:
#load in the data
file_path = "../../data/real/covtype_train.csv"
data = pd.read_csv(file_path)
dat_array = data.values[:]
N = dat_array.shape[0]
x = np.column_stack([np.ones(N), dat_array[:, 1:]])
y = dat_array[:,0]

#set up model parameters
dim = x.shape[1] 

#priors
mu_0 = np.zeros(dim) #prior mean
lambda_0 = 10.0*np.eye(dim)  #prior covariance matrix

In [4]:
#from sklearn.linear_model import LogisticRegression

#results from a fast MLE
#lr = LogisticRegression()
#lr.fit(dat_array[:, 1:], y)
#mle = np.concatenate((lr.intercept_,lr.coef_[0,:]))

#print("MLE estimates: ", "\n", mle, "\n")

### Set up sampling framework

In [5]:
# step-size
step_size = 10**(-6)
# batch sizes
n_batch = np.int64(N*np.array([0.01, 0.05, 0.1]))
# number of chains
N_rep = 11

### SGLD (0.01N)

In [6]:
n = n_batch[0]
burnin = np.int64(500/0.01)
Niter = 2*burnin

runtime_df=[]
samples_df = []
grads_df = []

In [7]:
for i in tqdm(range(N_rep), desc = "Number of chains run"):
    key, subkey = random.split(key)
    theta_0 = jnp.zeros(dim)
    samples, grads, runtime = sgld.sgld_sampler(subkey, gradf_0, gradf_i_batch, Niter, step_size, theta_0, x, y, n, replacement=True)
    runtime_df.append(runtime)
    samples_df.append(samples)
    grads_df.append(grads)

Number of chains run: 100%|██████████| 11/11 [1:29:21<00:00, 487.44s/it]


In [8]:
runtime_df = pd.DataFrame(np.column_stack(runtime_df))
runtime_df.to_csv("./out/lrb_sgld_1_runtime.csv", index=False)
samples_df = pd.DataFrame(np.column_stack(samples_df))
samples_df.to_csv("./out/lrb_sgld_1_samples.csv", index=False)
grads_df = pd.DataFrame(np.column_stack(grads_df))
grads_df.to_csv("./out/lrb_sgld_1_grads.csv", index=False)

### SGLD (0.05N)

In [9]:
n = n_batch[1]
burnin = np.int64(500/0.05)
Niter = 2*burnin

runtime_df=[]
samples_df = []
grads_df = []

In [10]:
for i in tqdm(range(N_rep), desc = "Number of chains run"):
    key, subkey = random.split(key)
    theta_0 = jnp.zeros(dim)
    samples, grads, runtime = sgld.sgld_sampler(subkey, gradf_0, gradf_i_batch, Niter, step_size, theta_0, x, y, n, replacement=True)
    runtime_df.append(runtime)
    samples_df.append(samples)
    grads_df.append(grads)

Number of chains run: 100%|██████████| 11/11 [32:16<00:00, 176.01s/it]


In [11]:
runtime_df = pd.DataFrame(np.column_stack(runtime_df))
runtime_df.to_csv("./out/lrb_sgld_5_runtime.csv", index=False)
samples_df = pd.DataFrame(np.column_stack(samples_df))
samples_df.to_csv("./out/lrb_sgld_5_samples.csv", index=False)
grads_df = pd.DataFrame(np.column_stack(grads_df))
grads_df.to_csv("./out/lrb_sgld_5_grads.csv", index=False)

### SGLD (0.1N)

In [12]:
n = n_batch[2]
burnin = np.int64(500/0.1)
Niter = 2*burnin

runtime_df=[]
samples_df = []
grads_df = []

In [13]:
for i in tqdm(range(N_rep), desc = "Number of chains run"):
    key, subkey = random.split(key)
    theta_0 = jnp.zeros(dim)
    samples, grads, runtime = sgld.sgld_sampler(subkey, gradf_0, gradf_i_batch, Niter, step_size, theta_0, x, y, n, replacement=True)
    runtime_df.append(runtime)
    samples_df.append(samples)
    grads_df.append(grads)

Number of chains run: 100%|██████████| 11/11 [27:48<00:00, 151.66s/it]


In [14]:
runtime_df = pd.DataFrame(np.column_stack(runtime_df))
runtime_df.to_csv("./out/lrb_sgld_10_runtime.csv", index=False)
samples_df = pd.DataFrame(np.column_stack(samples_df))
samples_df.to_csv("./out/lrb_sgld_10_samples.csv", index=False)
grads_df = pd.DataFrame(np.column_stack(grads_df))
grads_df.to_csv("./out/lrb_sgld_10_grads.csv", index=False)

### SGLD-PS (0.01N)

In [15]:
n = n_batch[0]
burnin = np.int64(500/0.01)
step_sgd = 1e-03
Niter = 2*burnin
runtime_df=[]
samples_df = []
grads_df = []

In [16]:
for i in tqdm(range(N_rep), desc = "Number of chains run"):
    key, subkey = random.split(key)
    theta_0 = jnp.zeros(dim)
    theta_hat, samples_sgd, runtime_sgd = sgd.adam(key, gradf_0, gradf_i_batch, burnin, theta_0, x, y, n, step_sgd, replacement=True)
    samples, grads, runtime = sgldps.sgldps_sampler(subkey, gradf_0, gradf_i_batch, burnin, step_size, theta_hat, theta_hat, x, y, n,  prob_type='approx')
        
    runtime_df.append(np.concatenate((runtime_sgd, runtime_sgd[burnin-1]+runtime)))
    samples_df.append(np.concatenate((samples_sgd, samples[1:]), axis=0))
    grads_df.append(grads)

Number of chains run: 100%|██████████| 11/11 [3:01:24<00:00, 989.53s/it]  


In [17]:
runtime_df = pd.DataFrame(np.column_stack(runtime_df))
runtime_df.to_csv("./out/lrb_sgldps_1_runtime.csv", index=False)
samples_df = pd.DataFrame(np.column_stack(samples_df))
samples_df.to_csv("./out/lrb_sgldps_1_samples.csv", index=False)
grads_df = pd.DataFrame(np.column_stack(grads_df))
grads_df.to_csv("./out/lrb_sgldps_1_grads.csv", index=False)

### SGLD-PS (0.05N)

In [18]:
n = n_batch[1]
burnin = np.int64(500/0.05)
Niter = 2*burnin
runtime_df=[]
samples_df = []
grads_df = []

In [19]:
for i in tqdm(range(N_rep), desc = "Number of chains run"):
    key, subkey = random.split(key)
    theta_0 = jnp.zeros(dim)
    theta_hat, samples_sgd, runtime_sgd = sgd.adam(key, gradf_0, gradf_i_batch, burnin, theta_0, x, y, n, step_sgd, replacement=True)
    samples, grads, runtime = sgldps.sgldps_sampler(subkey, gradf_0, gradf_i_batch, burnin, step_size, theta_hat, theta_hat, x, y, n,  prob_type='approx')
        
    runtime_df.append(np.concatenate((runtime_sgd, runtime_sgd[burnin-1]+runtime)))
    samples_df.append(np.concatenate((samples_sgd, samples[1:]), axis=0))
    grads_df.append(grads)

Number of chains run: 100%|██████████| 11/11 [1:44:17<00:00, 568.91s/it]


In [20]:
runtime_df = pd.DataFrame(np.column_stack(runtime_df))
runtime_df.to_csv("./out/lrb_sgldps_5_runtime.csv", index=False)
samples_df = pd.DataFrame(np.column_stack(samples_df))
samples_df.to_csv("./out/lrb_sgldps_5_samples.csv", index=False)
grads_df = pd.DataFrame(np.column_stack(grads_df))
grads_df.to_csv("./out/lrb_sgldps_5_grads.csv", index=False)

### SGLD-PS (0.1N)

In [21]:
n = n_batch[2]
burnin = np.int64(500/0.1)
Niter = 2*burnin
runtime_df=[]
samples_df = []
grads_df = []

In [22]:
for i in tqdm(range(N_rep), desc = "Number of chains run"):
    key, subkey = random.split(key)
    theta_0 = jnp.zeros(dim)
    theta_hat, samples_sgd, runtime_sgd = sgd.adam(key, gradf_0, gradf_i_batch, burnin, theta_0, x, y, n, step_sgd, replacement=True)
    samples, grads, runtime = sgldps.sgldps_sampler(subkey, gradf_0, gradf_i_batch, burnin, step_size, theta_hat, theta_hat, x, y, n,  prob_type='approx')
        
    runtime_df.append(np.concatenate((runtime_sgd, runtime_sgd[burnin-1]+runtime)))
    samples_df.append(np.concatenate((samples_sgd, samples[1:]), axis=0))
    grads_df.append(grads)

Number of chains run: 100%|██████████| 11/11 [1:35:35<00:00, 521.42s/it]


In [23]:
runtime_df = pd.DataFrame(np.column_stack(runtime_df))
runtime_df.to_csv("./out/lrb_sgldps_10_runtime.csv", index=False)
samples_df = pd.DataFrame(np.column_stack(samples_df))
samples_df.to_csv("./out/lrb_sgldps_10_samples.csv", index=False)
grads_df = pd.DataFrame(np.column_stack(grads_df))
grads_df.to_csv("./out/lrb_sgldps_10_grads.csv", index=False)